DB 초기화 프로그램

In [15]:
!pip install pymysql > /dev/null

from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [17]:
import json, pymysql
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

conn = pymysql.connect(**config)    # dictionary unpacking

In [18]:
import random
random.seed(20201)

product 테이블

In [19]:
# products 테이블 생성 sql
sql = """
    CREATE TABLE if NOT exists products(
        pid INT PRIMARY KEY AUTO_INCREMENT,
        pname VARCHAR(40) NOT NULL,
        pprice INT UNSIGNED NOT NULL,
        pcategory VARCHAR(20) NOT NULL,
        pcost INT UNSIGNED NOT NULL
    ) AUTO_INCREMENT=11;
"""

In [20]:
cur = conn.cursor()
cur.execute(sql)

0

In [21]:
camping_goods = ['텐트','타프','천막','침낭','매트','랜턴','취사용품',
                 '아이스박스','보조배터리','워터저그']
car_goods = ['블랙박스','내비게이션','하이패스','후방카메라','자동차TV',
             '카오디오','핸즈프리','헤드유닛','방진매트','윤활방청제']
health_goods = ['러닝머신','웨이트기구','복근운동기구','벨트','거꾸리',
                '훌라후프','트램펄린','스텝퍼','로잉머신','헬스사이클']
category_names = ['캠핑용품','자동차용품','헬스용품']

In [22]:
for item in camping_goods:
    price = random.randint(100,999) * 100
    rate = random.randint(85, 98)
    cost = int(round(price * rate / 1000) * 10)
    print(item, price, '캠핑', cost)


텐트 15000 캠핑 13050
타프 56900 캠핑 53490
천막 44900 캠핑 38160
침낭 44800 캠핑 40770
매트 34800 캠핑 31320
랜턴 27200 캠핑 23940
취사용품 55800 캠핑 53010
아이스박스 97800 캠핑 95840
보조배터리 83900 캠핑 79700
워터저그 53500 캠핑 52430


In [23]:
sql = "insert into products(pname,pprice,pcategory,pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([camping_goods, car_goods, health_goods]):
    for item in item_list:
        price = random.randint(100,999) * 100
        rate = random.randint(85, 98)
        cost = int(round(price * rate / 1000) * 10)
        cur.execute(sql, (item,price,category_names[i],cost))
conn.commit()

sales 테이블

sales( sid , sdate , scompany , spid , sunit

In [24]:
sql = """
    CREATE TABLE if NOT EXISTS sales(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT NULL,
        scompany VARCHAR(20) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY(spid) REFERENCES products(pid)
    ) AUTO_INCREMENT=101;
"""

In [25]:
cur = conn.cursor()
cur.execute(sql)

0

In [75]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
company_list = ['멀티캠퍼스','한경아카데미','표준협회','생산성본부','테크브루']
for i in range(10):
    date = f'2020-1-{random.randint(1,31)}'
    company = company_list[random.randint(0,4)]
    pid = random.randint(11,40)
    unit = random.randint(1,20)
    print(date,company,pid,unit)

2020-1-13 생산성본부 39 10
2020-1-27 테크브루 23 6
2020-1-21 한경아카데미 34 7
2020-1-24 멀티캠퍼스 31 11
2020-1-22 멀티캠퍼스 19 1
2020-1-6 표준협회 12 10
2020-1-2 표준협회 18 8
2020-1-3 생산성본부 26 5
2020-1-20 멀티캠퍼스 27 18
2020-1-16 생산성본부 39 14


In [76]:
sql = "insert into sales values(default,%s,%s,%s,%s);"
for month, day in calendar.items():
    for i in range(10):
        date = f'2020-{month}-{random.randint(1,day)}'
        company = company_list[random.randint(0,4)]
        pid = random.randint(11,40)
        unit = random.randint(1,20)
        cur.execute(sql, (date, company, pid, unit))
conn.commit()

In [77]:
sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""

In [78]:
cur = conn.cursor()
cur.execute(sql)

0

In [79]:
sql = 'select * from sales_book'
cur.execute(sql)
results = cur.fetchall()

In [80]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,107,2020-01-01,표준협회,11,텐트,캠핑용품,30300,18,27270,545400,54540
1,103,2020-01-03,한경아카데미,22,내비게이션,자동차용품,48200,9,46270,433800,17370
2,104,2020-01-04,테크브루,29,방진매트,자동차용품,23800,15,22370,357000,21450
3,108,2020-01-05,표준협회,16,랜턴,캠핑용품,90400,10,88590,904000,18100
4,102,2020-01-13,멀티캠퍼스,16,랜턴,캠핑용품,90400,10,88590,904000,18100


In [81]:
del df[3]
del df[0]

In [82]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-01,표준협회,텐트,캠핑용품,30300,18,27270,545400,54540
1,2020-01-03,한경아카데미,내비게이션,자동차용품,48200,9,46270,433800,17370
2,2020-01-04,테크브루,방진매트,자동차용품,23800,15,22370,357000,21450
3,2020-01-05,표준협회,랜턴,캠핑용품,90400,10,88590,904000,18100
4,2020-01-13,멀티캠퍼스,랜턴,캠핑용품,90400,10,88590,904000,18100


In [83]:
df.to_csv('sales_book.csv', index=False)

In [84]:
cur.close()
conn.close()